In [1]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from sklearn import metrics
import datetime
import matplotlib.pyplot as plt

try:
    from google.colab import drive

    drive.mount("/content/drive", force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    # Since finta not on Google CoLab.
    from finta import TA

    print("Note: not using Google CoLab")
    COLAB = False


def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res


def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)


def to_sequences(seq_size, obs, target_col):
    x = []
    y = []

    for i in range(len(obs) - seq_size):
        window = obs.iloc[i : (i + seq_size), target_col:].values
        after_window = obs.iloc[i + seq_size, target_col]
        window = [[x] for x in window]
        x.append(window)
        y.append(after_window)

    x_train = np.array(x)
    y_train = np.array(y)
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], x_train.shape[3]))

    return x_train, y_train


def backtest(
    pred,
    y_test,
    periods_per_day=10,
    capital=1_000_000,
    mult=10_000,
    contracts=1,
    idx=None,
):
    total = 0
    pnls = []
    counter = 0

    # TODO: Fix to correct date indexing.
    # idx_n = []

    for x in range(len(pred)):
        if pred[x] > 0.0:
            total += y_test[x] * mult * contracts
        elif pred[x] < 0.0:
            total -= y_test[x] * mult * contracts
        if counter % periods_per_day == 0:
            pnls.append(total)
            # idx_n.append(idx[counter])
        counter += 1

    rets = pd.DataFrame(data=pnls, columns=["cum_pnl"])  # , index=idx_n)
    rets["portfolio"] = rets["cum_pnl"] + capital
    rets["pct_pnl"] = rets["portfolio"].pct_change()
    rets = rets.dropna()

    return rets


def perf_summ(data: pd.DataFrame, adj: int = 12, title="Metric") -> pd.DataFrame:
    summary = pd.DataFrame(
        data=data.mean() * adj, index=[title], columns=["Annualized Return"]
    )
    summary["Annualized Volatility"] = data.std() * np.sqrt(adj)
    summary["Annualized Sharpe Ratio"] = (
        summary["Annualized Return"] / summary["Annualized Volatility"]
    )
    summary["Annualized Sortino Ratio"] = summary["Annualized Return"] / (
        (data[data < 0]).std() * np.sqrt(adj)
    )

    summary["Skewness"] = data.skew()
    summary["Kurtosis"] = data.kurtosis()
    summary["VaR (0.05)"] = data.quantile(0.05)
    summary["CVaR (0.05)"] = data[data <= data.quantile(0.05)].mean()
    summary["Min"] = data.min()
    summary["Max"] = data.max()

    wealth_index = 1000 * (1 + data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks) / previous_peaks

    summary["Max Drawdown"] = drawdowns.min()

    # TODO: Add absolute value/fix Calmar calculation.
    summary["Calmar Ratio"] = (data.mean() * adj) / drawdowns.min()

    return summary.T


def run_stepped(model, x_test, y_test, step_size, stop_at=None):
    stop_at = len(x_test) if stop_at is None else stop_at
    x_test = x_test[:stop_at]
    y_test = y_test[:stop_at]
    preds = []

    for i in range(0, stop_at, step_size):
        # Check if we reached end of data.
        if i + step_size >= stop_at:
            x_step = x_test[i:]
            y_step = y_test[i:]
        else:
            x_step = x_test[i : i + step_size]
            y_step = y_test[i : i + step_size]

        # Predict the data.
        pred = model.predict(x_step)

        # Append to output list.
        preds += pred[:, 0].tolist()

        # Re-fit the model.
        try:
            with tf.device("/device:GPU:0"):
                model.fit(x_step, y_step)
        except:
            model.fit(x_step, y_step)
    # At the end, return x/y
    preds_np = np.array(preds)
    return preds_np, y_test[:stop_at]


periods_per_day = 10
fn = "/content/drive/MyDrive/Colab Notebooks/data/qm_data_tufv.csv"
df = pd.read_csv(fn, sep=",", na_values=["-1"], index_col=False)

In [2]:
target_col = "target"

# Taking sin and cos of hour (from: https://github.com/nklingen/Transformer-Time-Series-Forecasting)
df["Date"] = pd.to_datetime(df["Date"])
df["CosHour"] = np.cos(2 * np.pi * df["Date"].dt.hour / 24)
df["SinHour"] = np.sin(2 * np.pi * df["Date"].dt.hour / 24)
col = df.pop(target_col)
df.insert(0, col.name, col)
df_f = df.drop(["Sum", "Date"], axis=1).dropna()

x_train, y_train = to_sequences(50, df_f, 0)

In [3]:
model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=1e-4)
)

model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

df = df_f.copy()

df_train = df.iloc[:6_000, :]
df_test = df.iloc[6_000:, :]

x_train = df_train.drop(target_col, axis=1)
y_train = df_train[target_col]

x_test = df_test.drop(target_col, axis=1)
y_test = df_test[target_col]

y_train = y_train.to_numpy().reshape((-1, 1))
y_test = y_test.to_numpy().reshape((-1, 1))

x_train, y_train = to_sequences(50, df_train, 0)
x_test, y_test = to_sequences(50, df_test, 0)

try:
    with tf.device("/device:GPU:0"):
        model.fit(
            x_train,
            y_train,
            epochs=20,
            validation_split=0.2,
            batch_size=64,
            callbacks=callbacks,
        )
except:
    model.fit(
        x_train,
        y_train,
        epochs=20,
        validation_split=0.2,
        batch_size=64,
        callbacks=callbacks,
    )

In [ ]:
x, y = run_stepped(
    model,
    x_test,
    y_test,
    step_size=20,
)

In [ ]:
rets = backtest(x, y, periods_per_day=10, mult=1, capital=3.27)
display(perf_summ(rets["pct_pnl"], adj=252, title="Window=50"))
agg_perf = perf_summ(rets["pct_pnl"], adj=252, title="Transformer")
plt.plot(rets["portfolio"])
plt.plot(rets["portfolio"].cummax())
plt.title("Original Feature Space")
plt.show()